In [1]:
import pandas as pd
import numpy as np

In [6]:
qb = pd.read_csv("finalized_QB.csv", index_col=0)


#### Useless variables
- Season 
- 

In [15]:
qb["Season"].unique()

array(['Regular Season'], dtype=object)

In [12]:
qb[['points']]

,points
0,33.56
1,14.44
2,23.32
3,25.84
4,17.52
5,0.00
6,20.56
7,23.46
8,32.90
9,30.94
